In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pandas as pd
sys.path.append('/Users/sheep/PyPathway/')
from pypathway.analysis.ora import Reactome, GO, KEGG, ORA
from pypathway.utils import ColorectalCancer, IdMapping, GMTUtils
from IPython.display import HTML
import json

In [3]:
# load a gmt file.
gmt = GMTUtils.parse_gmt_file("../../tests/gmt_file/h.all.v6.0.entrez.gmt")

In [5]:
# load the example
c = ColorectalCancer()

In [6]:
len(c.deg_list), len(c.background)

(5320, 17216)

In [10]:
res_h = ORA.run(c.deg_list, c.background, gmt)

In [11]:
res.table.head()

,name,mapped,number in study,p-value,fdr
0,HALLMARK_BILE_ACID_METABOLISM,104,46,2.793934e-03,4.656557e-03
1,HALLMARK_MYC_TARGETS_V1,173,48,8.377692e-01,9.106187e-01
2,HALLMARK_INFLAMMATORY_RESPONSE,184,68,4.550977e-02,6.692613e-02
3,HALLMARK_ESTROGEN_RESPONSE_LATE,191,100,4.735396e-10,2.630776e-09
4,HALLMARK_KRAS_SIGNALING_DN,184,43,9.907251e-01,9.992615e-01


In [12]:
res.plot()

In [10]:
# use kegg enrichment toward certain 

In [13]:
r_kg = KEGG.run(c.deg_list, c.background, 'hsa')

In [15]:
r_kg.plot()

In [14]:
# the Example of using the warpper of Reactome gene set enrichment analysis

In [16]:
sybs = [x[1][0] for x in IdMapping.convert(input_id=c.deg_list, organism='hsa', source='ENTREZID', target='SYMBOL') if x[1]]

In [17]:
len(sybs)

5320

In [18]:
# the input is a list of symbol
r = Reactome.run(sybs[:10], organism='Homo sapiens')

4


In [19]:
# the result
r.table

,name,dbId,found,p-value,fdr,species
0,Acetylation,156582,2,0.000722,0.031066,Homo sapiens
1,Defective SLC6A2 causes orthostatic intoleranc...,5619109,1,0.007301,0.153318,Homo sapiens
2,Amino acid transport across the plasma membrane,352230,2,0.012057,0.168804,Homo sapiens
3,Neurotransmitter uptake and metabolism In glia...,112313,1,0.024136,0.184060,Homo sapiens
4,Astrocytic Glutamate-Glutamine Uptake And Meta...,210455,1,0.024136,0.184060,Homo sapiens
5,BBSome-mediated cargo-targeting to cilium,5620922,1,0.056979,0.184060,Homo sapiens
6,HDL assembly,8963896,1,0.043037,0.184060,Homo sapiens
7,Platelet degranulation,114608,2,0.044107,0.184060,Homo sapiens
8,Response to elevated platelet cytosolic Ca2+,76005,2,0.048221,0.184060,Homo sapiens
9,Intrinsic Pathway of Fibrin Clot Formation,140837,1,0.056979,0.184060,Homo sapiens


In [20]:
# the overview
r.plot()

In [21]:
# make the assoc file using the id_mapping function

In [22]:
from pypathway.utils import IdMapping

In [23]:
r = IdMapping.convert_to_dict(input_id=c.background, source='ENTREZID', target="GO", organism='hsa')

In [25]:
#run
path = '/Users/sheep/goatools/'
rg = GO.run([str(x) for x in c.deg_list], [str(x) for x in c.background], r, obo=path + 'go-basic.obo')

load obo file /Users/sheep/goatools/go-basic.obo
/Users/sheep/goatools/go-basic.obo: fmt(1.2) rel(2017-04-21) 48,571 GO Terms
fisher module not installed.  Falling back on scipy.stats.fisher_exact


Propagating term counts to parents ..


15,253 out of 17,216 population items found in association
Calculating uncorrected p-values using fisher_scipy_stats
 5,104 out of  5,320 study items found in association
Running multitest correction: local bonferroni
Running multitest correction: local sidak
Running multitest correction: local holm
  15,732 GO terms are associated with 5,104 of 5,320 study items
  20,700 GO terms are associated with 15,253 of 17,216 population items


In [26]:
rg.plot()

In [45]:
rg = GO.run(path + 'data/study', path + 'data/population', path + 'data/association',
        obo=path + 'go-basic.obo')

load obo file /Users/sheep/goatools/go-basic.obo
/Users/sheep/goatools/go-basic.obo: fmt(1.2) rel(2017-04-21) 48,571 GO Terms
fisher module not installed.  Falling back on scipy.stats.fisher_exact


Propagating term counts to parents ..
terms not found: {'GO:0022627', 'GO:0022625'}


31,855 out of 33,239 population items found in association
Calculating uncorrected p-values using fisher_scipy_stats
   269 out of    276 study items found in association
Running multitest correction: local bonferroni
Running multitest correction: local sidak
Running multitest correction: local holm
  797 GO terms are associated with 269 of 276 study items
  6,120 GO terms are associated with 31,855 of 33,239 population items


In [48]:
rg.graph()

In [31]:
from pypathway.exportion import EnrichmentExport

In [57]:
c = EnrichmentExport.export([rg, rg])

In [52]:
pwd

'/Users/sheep/PyPathway/examples/analysis'

In [120]:
def enrichment_export(result_list, devide_dataset=False):
    layout = {}
    snapshot = []
    detail = []
    if devide_dataset:
        # devide dataset here
        pass
    else:
        layout = {'layer': 1, 'display': 'card'}
    for i, x in enumerate(result_list):
        detail.append({'bar': x.plot(data=True), 'overview': rg.graph()})
        snapshot.append({'snap': x.snapshot(), 'id': 'plot{}'.format(i), 'tags': x.overview()})
    return {"layout": layout, 'snapshot': snapshot, 'detail': detail}

def generate_page_by_config(config):
    # first generate the card page
    tmp_card = '/Users/sheep/WebstormProjects/jinja_template/card.html'
    tmp_detail = '/Users/sheep/WebstormProjects/jinja_template/enrichment_display.html'
    with open(tmp_card) as fp:
        con = fp.read()
    t = Template(con)
    items = []
    # generate the card layout
    for i, x in enumerate(config['snapshot']):
        if i % 2 == 0:
            current = {'has1': True, 'has2': False}
            current['F'] = {'id': x['id'], 'tags': x['tags']}
            if i == len(config['snapshot']) - 1:
                items.append(current)
        else:
            current['has2'] = True
            current['S'] = {'id': x['id'], 'tags': x['tags']}
            items.append(current)
    with open('/Users/sheep/WebstormProjects/jinja_template/cd1.html', 'w') as fp:
        fp.write(t.render(items=items))
    # generate the snapshot config
    snapshot_config = {}
    snapshot_config_path = "/Users/sheep/WebstormProjects/jinja_template/data/card.json"
    for x in config['snapshot']:
        snapshot_config[x['id']] = {'type': 'chart', 'snap': x['snap'], 'detail': {"method": "ORA"}}
    print(snapshot_config)
    with open(snapshot_config_path, 'w') as fp:
        json.dump(snapshot_config, fp)

In [124]:
r = enrichment_export([rg, rg, rg])
generate_page_by_config(r)

{'plot0': {'snap': {'yAxis': {'splitLine': {'show': False}, 'data': ['cytoplasmic part', 'cellular_component', 'intracellular part', 'cell part', 'molecular_function', 'biological_process', 'organelle', 'single-organism process'], 'type': 'category'}, 'series': [{'data': [322.17044167570776, 316.2821667582666, 277.49105930589093, 261.1753088778956, 232.213038547228, 231.38440696554, 197.55866244673427, 195.24529636535036], 'name': '-lg(p_bonferroni)', 'type': 'bar'}], 'color': ['#F2F2F2'], 'xAxis': {'lineColor': 'transparent', 'splitLine': {'show': False}, 'type': 'value', 'boundaryGap': [0, 0.01], 'name': '-lg(p_bonferroni)'}, 'tooltip': {'trigger': 'axis', 'axisPointer': {'type': 'shadow'}}, 'grid': {'bottom': '3%', 'top': '5%', 'containLabel': True, 'left': '3%', 'right': '12%'}}, 'detail': {'method': 'ORA'}, 'type': 'chart'}, 'plot1': {'snap': {'yAxis': {'splitLine': {'show': False}, 'data': ['cytoplasmic part', 'cellular_component', 'intracellular part', 'cell part', 'molecular_fu

In [97]:
from pypathway.pathviz.utils import plot_json

In [114]:
plot_json(rg.snapshot())

In [71]:
def write_data(template_path, out_put_path, json_path):
    with open(template_path) as fp:
        con = fp.read()
    t = Template(con)
    with open(out_put_path, 'w') as fp:
        fp.write(t.render(table=tb, title="Gene ontology enrichment"))
    with open(json_path, 'w') as fp:
        json.dump({'bar': bar, 'overview': overview}, fp)

In [79]:
write_data("/Users/sheep/WebstormProjects/jinja_template/enrichment_display.html",
          "/Users/sheep/WebstormProjects/jinja_template/enrichment_display2.html",
          "/Users/sheep/WebstormProjects/jinja_template/data/data.json")

In [64]:
rg.table.head()

,GO,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm
0,GO:0006464,BP,e,cellular protein modification process,33/276,1727/33239,0.000008,n.a.,33,0.0505,0.0492,0.0505
1,GO:0036211,BP,e,protein modification process,33/276,1727/33239,0.000008,n.a.,33,0.0505,0.0492,0.0505
2,GO:0006468,BP,e,protein phosphorylation,22/276,922/33239,0.000011,n.a.,22,0.0660,0.0643,0.0659
3,GO:0016310,BP,e,phosphorylation,22/276,996/33239,0.000035,n.a.,22,0.2120,0.2070,0.2120
4,GO:0043412,BP,e,macromolecule modification,33/276,1877/33239,0.000057,n.a.,33,0.3510,0.3420,0.3510


In [20]:
print()

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>GO</th>
      <th>NS</th>
      <th>enrichment</th>
      <th>name</th>
      <th>ratio_in_study</th>
      <th>ratio_in_pop</th>
      <th>p_uncorrected</th>
      <th>depth</th>
      <th>study_count</th>
      <th>p_bonferroni</th>
      <th>p_sidak</th>
      <th>p_holm</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>4067</th>
      <td>GO:0003824</td>
      <td>MF</td>
      <td>e</td>
      <td>catalytic activity</td>
      <td>106/276</td>
      <td>7723/33239</td>
      <td>1.600000e-08</td>
      <td>n.a.</td>
      <td>106</td>
      <td>0.000098</td>
      <td>0.000095</td>
      <td>0.000098</td>
    </tr>
    <tr>
      <th>4068</th>
      <td>GO:0016740</td>
      <td>MF</td>
      <td>e</td>
      <td>transferase activity</td>
      <td>54/276</td>
      <td>2976/33239</td>
      <td>5.430000e-08</td>
      <td>n.a.</td>
      <td>54</td>
      <td>0.000332

In [48]:
rg.plot

In [10]:
# view the significant gene set in the network view
rg.overview()